# Advanced Speed and Accuracy Topics

## Calling a C function natively

## OpenCL

## Benchmarking, Timing, Profiling

In [ ]:
@time

In [ ]:
using BenchmarkTools
X, Y = rand(1000,1000), rand(1000,1000);
@benchmark max(abs(X), abs(Y))

## Debugging

In [ ]:
using Gallium